In [0]:
!pip install catboost

     |████████████████████████████████| 61.9MB 886kB/s 


In [0]:
import numpy as np
RANDOM_STATE = 2019
np.random.seed(RANDOM_STATE) 

from google.colab import drive
drive.mount('/gdrive')

import pandas as pd
import time
import lightgbm as lgb

import seaborn as sns
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import KFold
from scipy import stats
from plotly.offline import init_notebook_mode, iplot
from plotly import graph_objs as go

from catboost import CatBoostRegressor
from numba import jit

import gc
import os

def inverse_boxcox(y, lambda_):
    return np.exp(y) if lambda_ == 0 else np.exp(np.log(lambda_ * y + 1) / lambda_)

os.chdir("/gdrive/My Drive/Colab Notebooks/dacon")


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
resampled = pd.read_csv('./data/resampled_0825.csv',parse_dates=['date'])
resampled.drop('holiday',axis=1,inplace=True)
resampled.shape

naive_store_info = pd.read_csv('data/comp_9th/naive_store_info.csv')
naive_store_info = naive_store_info[['store_id','type_bus','high_region']]
resampled = resampled.merge(naive_store_info, on=['store_id'])


In [0]:
for_reprod = resampled.copy()

In [0]:
resampled = for_reprod.copy()

In [0]:
resampled.drop(['cycle_idx_week1','cycle_idx_week4','cycle_idx_week8','hday_name','is_weekend','is_holiday','is_weekend','cur_zero'], axis=1,inplace = True)

In [0]:
# resampled = resampled[(resampled.Dayofweek == 3) | (resampled.Dayofweek == 0) ]

temp = resampled[['date','store_id','Dayofweek','y']].copy()

def modify_y(dow):
    if dow == 3:
        return 0
    elif dow < 3:
        i = 3 - dow
        return i
    else:
        i = 10 - dow
        return i
temp['date_sum'] = temp['Dayofweek'].apply(modify_y)
temp['date_sum'] = pd.to_timedelta(temp['date_sum'],unit='d')
temp['date_thurs'] = temp['date'] + temp['date_sum']

thurs_y = temp[temp.Dayofweek == 3 ][['date','store_id','y']]
thurs_y.columns = ['date_thurs','store_id','new_y']

temp = temp.merge(thurs_y, how = 'left' , on = ['date_thurs','store_id' ]) 
temp = temp[['date','store_id','new_y']]
resampled = resampled.merge(temp,how = 'left', on= ['date','store_id'] )

resampled.drop('y',axis=1,inplace=True)
resampled = resampled.rename(columns={'new_y':'y'})
del temp
import gc
gc.collect()

63

In [0]:
target = 'y'
cat_vars = ['store_id',  'Month', 'Week', 'Day', 'Dayofweek','Dayofyear',
            'weekofyear', 'type_bus', 'high_region']  #, 'cycle_idx_week4', 'cycle_idx_week8']

cont_vars = [cols for cols in resampled.columns.values if cols not in cat_vars+[target,'date']]

ratios = [i for i in cont_vars if "ratio" in i]
non_ratios = [i for i in cont_vars if i not in ratios]




In [0]:
for v in non_ratios:
    resampled[v][resampled[v] < 0] = 0
resampled[ratios] = resampled[ratios].replace(np.inf, 1e+17)
resampled[ratios] = resampled[ratios].replace(-np.inf, -1e+17)

resampled[non_ratios] = resampled[non_ratios].fillna(value=-999)
resampled[ratios] = resampled[ratios].fillna(value=-999)

print(resampled[cont_vars].columns[resampled[cont_vars].isna().any()].tolist())

#resampled = resampled.set_index('date')
resampled[cat_vars] = resampled[cat_vars].fillna(value='-999')

from sklearn.preprocessing import LabelEncoder
encoders = {}

for v in cat_vars:
    le = LabelEncoder()
    le.fit(resampled[v].values)
    encoders[v] = le
    resampled.loc[:, v] = le.transform(resampled[v].values)
    # print('{0}: {1}'.format(v, le.classes_))
gc.collect()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



[]


21

In [0]:
def train_4_submit(df,t,to_submit):
    file_type = ""
    if t == 1:
        file_type = "w0_"
    if t == 2:
        file_type = "existing_"

    resampled = df.copy()
    del df
    gc.collect()
    cat_vars = ['store_id',  'Month', 'Week', 'Day', 'Dayofweek','Dayofyear',
            'weekofyear', 'type_bus', 'high_region'] 
    cont_vars = [cols for cols in resampled.columns.values if cols not in cat_vars+['y','date']]

    #resampled[cat_vars] = resampled[cat_vars].fillna(-999)
    resampled[cat_vars] = resampled[cat_vars].astype('int')

    #resampled[cont_vars] = resampled[cont_vars].fillna(-999)
    resampled[cont_vars] = resampled[cont_vars].astype('float32')

    if to_submit:
        train_df = resampled[resampled.date <= pd.to_datetime('2018-11-29')]
        train_df = train_df[train_df.y > 0]
        test_df = resampled[(resampled.date <= pd.to_datetime('2019-02-28')) &
                      (resampled.date >= pd.to_datetime('2019-02-22')) ]
    else:
        train_df = resampled[resampled.date <= pd.to_datetime('2018-11-28')]
        train_df = train_df[train_df.y > 0]
        test_df = resampled[resampled.date  == pd.to_datetime('2018-11-29') ]

    X = train_df[cont_vars+cat_vars]
    y = train_df['y']
    target, lambda_prophet = stats.boxcox(train_df['y'] + 1)

    X_test = test_df[cont_vars+cat_vars]
    all_y = target 

    oof_preds = np.zeros([X.shape[0]])
    sub_preds = np.zeros([X_test.shape[0]])

    ITERATION = 5000
    N_SPLIT = 3
    feature_importance_df = pd.DataFrame()
    model_dir = "./submit_0821/cat_models/" 
    folds = KFold(n_splits=N_SPLIT, shuffle=True, random_state=RANDOM_STATE)
    params_cat = {
        'learning_rate':0.10,
        'iterations': ITERATION,
        'max_ctr_complexity': 9,
        'random_seed': RANDOM_STATE,
        'od_type': 'Iter',
        'od_wait': 100,
        'verbose': 500,
        'depth': 10,
        'l2_leaf_reg': 8,
        'thread_count': 4
    }

    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(X, target)):
        start_time = time.time()
        train_x, train_y = X.iloc[train_idx], target[train_idx]
        valid_x, valid_y = X.iloc[valid_idx], target[valid_idx]
        
        cat_model = CatBoostRegressor(eval_metric='MAE', **params_cat, task_type='GPU')
        cat_model.fit(
            train_x, 
            train_y, 
            eval_set=(valid_x, valid_y),
            cat_features=[X.columns.get_loc(c) for c in cat_vars],
            use_best_model=True
        )
        
        oof_preds[valid_idx] = cat_model.predict(valid_x)
        sub_preds[:] += cat_model.predict(X_test) / folds.n_splits
        valid_idx += 1

        # cat_model.save_model(model_dir+file_type+str(n_fold)+'_cat_model.json',
        #                       format = "json")

        importance_df = pd.DataFrame()
        importance_df['feature'] = X.columns
        importance_df['importance'] = cat_model.feature_importances_
        importance_df['fold'] = n_fold + 1
        
        feature_importance_df = pd.concat([feature_importance_df, importance_df], axis=0)
        print("--- %s seconds ---" % (time.time() - start_time))

        del cat_model
        gc.collect()

    sub_preds = inverse_boxcox(sub_preds , lambda_prophet) - 1
    oof_preds = inverse_boxcox(oof_preds , lambda_prophet) - 1

    e = 2 * abs(all_y - oof_preds) / ( abs(all_y)+abs(oof_preds) )
    e = e.mean()
    print('Full validation score Re-Box Cox Transformation is %.4f' %e)

    gc.collect()
    return sub_preds, feature_importance_df


In [0]:
import gc
gc.collect()

0

In [0]:
sub_pred1, f_i_df1 = train_4_submit(resampled,1,True)

- ref:
    - https://www.kaggle.com/slivka83/eda-futures-lightgbm-catboost#4.-Model